In [1]:
%pip install -e git+https://github.com/UN-GCPDS/python-gcpds.MI_prediction.git#egg=MI_prediction

Obtaining MI_prediction from git+https://github.com/UN-GCPDS/python-gcpds.MI_prediction.git#egg=MI_prediction
  Updating ./src/mi-prediction clone
  Running command git fetch -q --tags
  Running command git reset --hard -q b8bdb239a44e2d12357e55353110a5ca8de7e696
  Preparing metadata (setup.py) ... done
  Attempting uninstall: MI_prediction
    Found existing installation: MI-prediction 0.1
    Uninstalling MI-prediction-0.1:
      Successfully uninstalled MI-prediction-0.1
  Running setup.py develop for MI_prediction
Note: you may need to restart the kernel to use updated packages.


In [3]:
from MI_prediction.Utils.Datasets import DataLoader, DataLoader_Rest, get_epochs, get_runs
from MI_prediction.Datasets import Cho2017_Rest, BNCI2014001_Rest
from MI_prediction.Utils.Preprocess import filterbank_preprocessor, filterbank
from braindecode.preprocessing.preprocess import exponential_moving_standardize, preprocess, Preprocessor, scale

import numpy as np
import matplotlib.pyplot as plt

from sklearn.pipeline import Pipeline
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.model_selection import train_test_split,StratifiedKFold,cross_val_score
from mne.decoding import CSP
from sklearn.preprocessing import StandardScaler
from mne.preprocessing import compute_current_source_density

import copy

# Lawhern2018 sklearn

## BCI2a

### Load data

In [353]:
dl = DataLoader(dataset_name="BNCI2014001")
dl.load_data(subject_ids=[3])

48 events found
Event IDs: [1 2 3 4]
48 events found
Event IDs: [1 2 3 4]
48 events found
Event IDs: [1 2 3 4]
48 events found
Event IDs: [1 2 3 4]
48 events found
Event IDs: [1 2 3 4]
48 events found
Event IDs: [1 2 3 4]
48 events found
Event IDs: [1 2 3 4]
48 events found
Event IDs: [1 2 3 4]
48 events found
Event IDs: [1 2 3 4]
48 events found
Event IDs: [1 2 3 4]
48 events found
Event IDs: [1 2 3 4]
48 events found
Event IDs: [1 2 3 4]


In [355]:
def filterbank_preprocessor(freqs):
    FB = {}
    for freq in freqs:
        print('band to filter: {} Hz'.format(freq))
        FB[str(freq[0])+'_'+str(freq[1])] = Preprocessor('filter', l_freq=freq[0], h_freq=freq[1])
    return FB


In [356]:
fb = filterbank_preprocessor([(4.,38.)])

band to filter: (4.0, 38.0) Hz


In [357]:
fb

{'4.0_38.0': <braindecode.preprocessing.preprocess.Preprocessor at 0x2a49e4ee0>}

In [358]:
def filterbank(ds, preprocess = [], filters = [], standarization = [], channels_prep = []):
    ds_filt = []
    for i in filters.keys():
        d_tmp = copy.deepcopy(ds)
        preprocessors = preprocess + channels_prep + [filters[i]] + standarization
        d_tmp.preprocess_data(preprocessors=preprocessors)
        ds_filt.append(d_tmp)
    return ds_filt

In [359]:
preprocessors = [
        Preprocessor('pick_types', eeg=True, meg=False, stim=False),  # Keep EEG sensors
        Preprocessor(scale, factor=1e6, apply_on_array=True),  # Convert from V to uV
        #Preprocessor('filter', l_freq=1., h_freq=None)  # Bandpass filter
        ]

Ch_prep = [
        Preprocessor(compute_current_source_density,copy=False,apply_on_array=False)
        ]

In [360]:
Ch_prep

In [361]:
ds_f = filterbank(dl, preprocess=preprocessors, filters=fb, channels_prep=[])

Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 4 - 38 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 4.00
- Lower transition bandwidth: 2.00 Hz (-6 dB cutoff frequency: 3.00 Hz)
- Upper passband edge: 38.00 Hz
- Upper transition bandwidth: 9.50 Hz (-6 dB cutoff frequency: 42.75 Hz)
- Filter length: 413 samples (1.652 sec)

Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 4 - 38 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 4.00
- Lower transition bandwidth: 2.00 Hz (-6 dB cutoff frequency: 3.00 Hz)
- Upper passband edge: 

/Users/julian/miniforge3/envs/MI_pred/lib/python3.9/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function scale is deprecated; will be removed in 0.7.0. Use numpy.multiply instead.
  warnings.warn(msg, category=FutureWarning)


Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 4 - 38 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 4.00
- Lower transition bandwidth: 2.00 Hz (-6 dB cutoff frequency: 3.00 Hz)
- Upper passband edge: 38.00 Hz
- Upper transition bandwidth: 9.50 Hz (-6 dB cutoff frequency: 42.75 Hz)
- Filter length: 413 samples (1.652 sec)

Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 4 - 38 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 4.00
- Lower transition bandwidth: 2.00 Hz (-6 dB cutoff frequency: 3.00 Hz)
- Upper passband edge: 

In [362]:
ds_f

In [363]:
trials = [tr.get_trials(start_offset=[0], end_offset=[0]) for tr in ds_f]

Used Annotations descriptions: ['feet', 'left_hand', 'right_hand', 'tongue']
Adding metadata with 4 columns
48 matching events found
No baseline correction applied
0 projection items activated
Using data from preloaded Raw for 48 events and 1000 original time points ...
0 bad epochs dropped
Used Annotations descriptions: ['feet', 'left_hand', 'right_hand', 'tongue']
Adding metadata with 4 columns
48 matching events found
No baseline correction applied
0 projection items activated
Using data from preloaded Raw for 48 events and 1000 original time points ...
0 bad epochs dropped
Used Annotations descriptions: ['feet', 'left_hand', 'right_hand', 'tongue']
Adding metadata with 4 columns
48 matching events found
No baseline correction applied
0 projection items activated
Using data from preloaded Raw for 48 events and 1000 original time points ...
0 bad epochs dropped
Used Annotations descriptions: ['feet', 'left_hand', 'right_hand', 'tongue']
Adding metadata with 4 columns
48 matching even

In [364]:
trials

[{'win_0': <braindecode.datasets.base.BaseConcatDataset at 0x2acdf93d0>}]

In [365]:
epochs = [get_epochs(epoch['win_0'].split('session')['session_T']) for epoch in trials]

In [366]:
X = np.concatenate([np.expand_dims(trial[0],axis=3) for trial in epochs], axis=-1,dtype=np.float64)
y = epochs[0][1]

In [367]:
X[0,0,1]

array([-3.04061174])

In [368]:
from sklearn.base import BaseEstimator, TransformerMixin

In [369]:
class FBCSP(TransformerMixin,BaseEstimator):
    def __init__(self, n_components=8, reg=None, log=True, norm_trace=False):
        self.n_components = n_components
        self.reg = reg
        self.log = log
        self.norm_trace = norm_trace
    
    def fit(self, X, y):
        self.csp_mdl = CSP(n_components=self.n_components, reg=self.reg, log=self.log, norm_trace=self.norm_trace)
        self.mdls = [self.csp_mdl.fit(X[:,:,:,i], y) for i in range(X.shape[-1])]
        return self
    
    def transform(self, X):
        return np.concatenate([self.mdls[i].transform(X[:,:,:,i]) for i in range(len(self.mdls))],axis=-1)
    


In [370]:
idx_l = (y==0)+(y==1)
X = X[idx_l,:,:,:]
y = y[idx_l]

In [371]:
fbcsp = FBCSP(n_components=8,norm_trace=True, reg='empirical')

In [372]:
fbcsp.fit(X, y)

Computing rank from data with rank=None
    Using tolerance 52 (2.2e-16 eps * 22 dim * 1.1e+16  max singular value)
    Estimated rank (mag): 22
    MAG: rank 22 computed from 22 data channels with 0 projectors
Reducing data rank from 22 -> 22
Estimating covariance using EMPIRICAL
Done.
Computing rank from data with rank=None
    Using tolerance 46 (2.2e-16 eps * 22 dim * 9.4e+15  max singular value)
    Estimated rank (mag): 22
    MAG: rank 22 computed from 22 data channels with 0 projectors
Reducing data rank from 22 -> 22
Estimating covariance using EMPIRICAL
Done.


FBCSP(norm_trace=True, reg='empirical')

In [373]:
fbcsp.transform(X)

array([[5.9159225 , 5.70788447, 6.69524944, ..., 6.52846144, 6.34967594,
        6.42902798],
       [5.79297918, 6.23689518, 6.6830306 , ..., 6.83911074, 6.54311831,
        6.63190098],
       [5.87515531, 6.70162768, 6.63076426, ..., 6.68731102, 6.53610182,
        6.60167974],
       ...,
       [7.26605938, 6.5153053 , 6.61860488, ..., 6.92618248, 6.88718037,
        7.0251057 ],
       [6.62697308, 7.420823  , 6.96165135, ..., 6.85600663, 7.10572645,
        7.02750737],
       [6.9138421 , 6.49293496, 6.72358172, ..., 6.82053775, 6.94676154,
        6.803494  ]])

### CSP + LDA

In [394]:
preds = []
y_true = []
acc = []
c = 0
seed = 1234

skf = StratifiedKFold(n_splits=4)

lda = LinearDiscriminantAnalysis()
#csp = CSP(n_components=8, reg=None, log=True, norm_trace=False)
fbcsp = FBCSP(n_components=8,norm_trace=False)
scaler = StandardScaler()
clf = Pipeline([('CSP', fbcsp),('Scaler',scaler), ('LDA', lda)])

scores = cross_val_score(clf, X, y, cv=skf, n_jobs=-1,verbose=1)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.


Computing rank from data with rank=None
Computing rank from data with rank=None
Computing rank from data with rank=None
    Using tolerance 43 (2.2e-16 eps * 22 dim * 8.8e+15  max singular value)
    Estimated rank (mag): 22
    MAG: rank 22 computed from 22 data channels with 0 projectors
    Using tolerance 45 (2.2e-16 eps * 22 dim * 9.3e+15  max singular value)
    Estimated rank (mag): 22
    MAG: rank 22 computed from 22 data channels with 0 projectors
    Using tolerance 45 (2.2e-16 eps * 22 dim * 9.1e+15  max singular value)
Reducing data rank from 22 -> 22
    Estimated rank (mag): 22
    MAG: rank 22 computed from 22 data channels with 0 projectors
Estimating covariance using EMPIRICAL
Reducing data rank from 22 -> 22
Estimating covariance using EMPIRICAL
Done.
Done.
Reducing data rank from 22 -> 22
Estimating covariance using EMPIRICAL
Done.
Computing rank from data with rank=None
Computing rank from data with rank=None
Computing rank from data with rank=None
Computing rank f

[Parallel(n_jobs=-1)]: Done   4 out of   4 | elapsed:    0.5s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done   4 out of   4 | elapsed:    0.5s finished


In [395]:
scores

array([1.        , 0.97222222, 0.91666667, 0.88888889])

In [396]:
np.mean(scores)

0.9444444444444444